# Introduction
**Nama** : Dafa Wiratama
**Program** : Data Science
**Batch** : Batch 09

## Problems

Kamu adalah salah satu tim relawan back office penanganan COVID-19 di Indonesia. Kamu diminta untuk memperoleh data baru dari dataset yang sudah disediakan. Kerjakan dan jawab pertanyaan-pertanyaan berikut. (**Dataset yang disediakan merupakan jumlah kasus kumulatif bukan harian**)
1. Menggunakan Pandas query, ambil data kasus kumulatif Covid-19 tanggal 20 Januari 2022 untuk negara-negara dengan latitude di bawah 0. Negara mana yang paling tinggi jumlah kumulatif kasus Covidnya per tanggal tersebut? Jika bukan Indonesia, bandingkan jumlah kumulatif kasus negara tersebut dengan Indonesia per tanggal yang sama!
2. Buatlah fungsi untuk menghitung jumlah kasus harian Covid-19! (Hint: Kamu bisa menggunakan .diff() untuk menghitung perbedaan tiap baris data/menghitung jumlah kasus harian dan .transpose() untuk mengubah kolom menjadi baris atau sebaliknya)
3. Pada tanggal berapa puncak tertinggi kasus harian Covid-19 di Indonesia? bandingkan dengan tanggal 20 Januari 2022, apakah mengalami penurunan atau kenaikan jumlah kasus?
4. Gunakan for loop dan conditional if untuk membuat sebuah list yang berisikan jumlah kasus harian Covid-19 dengan ketentuan hanya mengambil data Indonesia saja dan jumlah kasus di bawah 10000. Berapa jumlah hari dengan kasus di bawah 10000?

# Library Import

in this assignment, we will be using the following libraries:
- pandas

In [1]:
import pandas as pd

# Dataset Loading

In this section we will perform dataset loading into our pandas dataframe.

In [2]:
url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
df = pd.read_csv(url)

# Dataset Exploration
From the dataset, we can see that there are many countries in the dataset. and some columns are not needed or at least in this given form.

### Dataset Inspection
by looking at the sample from the dataset we can grasp the following:
- there are `Province/State` column in the dataset
    but there are some null values in the dataset
    for this task we will leave it be because wa cannot replace it with other values because we don't know the value of the Province/State the sample was taken from
- there are `Country/Region` column in the dataset
    from shallow investigation we can assume this column is unique, and we can use it as a key to join the dataset
- there are `Lat` and `Long` columns in the dataset
    this column we will use it to query the data from the dataset by geographical location
- the rest of the columns are not needed in column, but we will pivot it to make it easier to work with


In [3]:
df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/10/22,2/11/22,2/12/22,2/13/22,2/14/22,2/15/22,2/16/22,2/17/22,2/18/22,2/19/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,169448,169940,170152,170604,171246,171422,171519,171673,171857,171931
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,267020,267020,267551,268008,268304,268491,268940,269301,269601,269904
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,260191,260723,261226,261752,262165,262570,262994,263369,263685,263936
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,37074,37140,37140,37140,37277,37361,37452,37522,37589,37589
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,98474,98501,98514,98514,98514,98555,98568,98585,98605,98617


## Dataset Cleaning
let's start by dividing our dataset into two parts.
1. We will use the first part to hold our header
2. the second part to hold our historical data.
3. we will not perform any nan value replacement because we don't know the value of the Province/State the sample was taken from

In [4]:
df["Identifier"] = [f"{df['Country/Region'][i]}-{df['Province/State'][i]}" for i in range(len(df))]

### Dataset Header
in this frame we only need the `Country/Region`, `Province/State`, `Lat` and `Long` column.

In [5]:
df_header = df[["Identifier", "Country/Region", "Province/State", "Lat", "Long"]]
df_header

,Identifier,Country/Region,Province/State,Lat,Long
0,Afghanistan-nan,Afghanistan,NaN,33.939110,67.709953
1,Albania-nan,Albania,NaN,41.153300,20.168300
2,Algeria-nan,Algeria,NaN,28.033900,1.659600
3,Andorra-nan,Andorra,NaN,42.506300,1.521800
4,Angola-nan,Angola,NaN,-11.202700,17.873900
...,...,...,...,...,...
279,West Bank and Gaza-nan,West Bank and Gaza,NaN,31.952200,35.233200
280,Winter Olympics 2022-nan,Winter Olympics 2022,NaN,39.904200,116.407400
281,Yemen-nan,Yemen,NaN,15.552727,48.516388
282,Zambia-nan,Zambia,NaN,-13.133897,27.849332


### Dataset Historical Data
In this frame we will transform the data into a format that we can use.
in this case historical data, so we will use the `Country/Region` as  column name and the rest of the columns as the row.
and we will parse the index of dataframe to be the date.

In [6]:
df_history_cumsum = df.iloc[:, 4:-1].T
df_history_cumsum.index = pd.to_datetime(df_history_cumsum.index)
df_history_cumsum.columns = df["Identifier"].values
df_history_cumsum

,Afghanistan-nan,Albania-nan,Algeria-nan,Andorra-nan,Angola-nan,Antarctica-nan,Antigua and Barbuda-nan,Argentina-nan,Armenia-nan,Australia-Australian Capital Territory,...,Uruguay-nan,Uzbekistan-nan,Vanuatu-nan,Venezuela-nan,Vietnam-nan,West Bank and Gaza-nan,Winter Olympics 2022-nan,Yemen-nan,Zambia-nan,Zimbabwe-nan
2020-01-22,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
2020-01-24,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
2020-01-25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
2020-01-26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-15,171422,268491,262570,37361,98555,11,7342,8766174,408381,42720,...,788676,234063,7,506977,2572087,618527,505,11699,309870,231603
2022-02-16,171519,268940,262994,37452,98568,11,7395,8783208,410155,43246,...,795316,234359,7,508042,2606824,622175,507,11707,310155,232213
2022-02-17,171673,269301,263369,37522,98585,11,7395,8799858,411878,43768,...,800833,234600,7,508968,2643024,625028,507,11718,310474,232598
2022-02-18,171857,269601,263685,37589,98605,11,7400,8815247,413295,44100,...,808012,234872,7,509947,2685463,627956,508,11731,310764,233030


### Dataset Historical Delta
in this frame we will create the calculation of the delta/difference between the date on each Country.

In [7]:
df_history_delta = df_history_cumsum.diff()
df_history_delta

,Afghanistan-nan,Albania-nan,Algeria-nan,Andorra-nan,Angola-nan,Antarctica-nan,Antigua and Barbuda-nan,Argentina-nan,Armenia-nan,Australia-Australian Capital Territory,...,Uruguay-nan,Uzbekistan-nan,Vanuatu-nan,Venezuela-nan,Vietnam-nan,West Bank and Gaza-nan,Winter Olympics 2022-nan,Yemen-nan,Zambia-nan,Zimbabwe-nan
2020-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
2020-01-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-15,176.0,187.0,405.0,84.0,41.0,0.0,11.0,18573.0,1307.0,557.0,...,6120.0,302.0,0.0,1030.0,31814.0,4681.0,4.0,20.0,291.0,0.0
2022-02-16,97.0,449.0,424.0,91.0,13.0,0.0,53.0,17034.0,1774.0,526.0,...,6640.0,296.0,0.0,1065.0,34737.0,3648.0,2.0,8.0,285.0,610.0
2022-02-17,154.0,361.0,375.0,70.0,17.0,0.0,0.0,16650.0,1723.0,522.0,...,5517.0,241.0,0.0,926.0,36200.0,2853.0,0.0,11.0,319.0,385.0
2022-02-18,184.0,300.0,316.0,67.0,20.0,0.0,5.0,15389.0,1417.0,332.0,...,7179.0,272.0,0.0,979.0,42439.0,2928.0,1.0,13.0,290.0,432.0


# Dataset Visualization
After trying to understand the dataset, we can see that it is impossible to visualize the data in a meaningful way.
because the data is too large.

In [8]:
# df_history_delta.plot()

## Utility Functions
this function will be used to format numerical data into a readable format.

In [9]:
def formatter(x):
    return "{:,}".format(x)

# Question 1
Menggunakan Pandas query, ambil data kasus kumulatif Covid-19 tanggal 20 Januari 2022 untuk negara-negara dengan latitude di bawah 0. Negara mana yang paling tinggi jumlah kumulatif kasus Covidnya per tanggal tersebut? Jika bukan Indonesia, bandingkan jumlah kumulatif kasus negara tersebut dengan Indonesia per tanggal yang sama!

## Answer :

Negara yang paling tinggi jumlah kumulatif kasus Covidnya per tanggal tersebut adalah 'Brazil' dengan jumlah kasus kumulatif Covid-19 23,595,178
Sementara negara Indonesia paling tinggi jumlah kumulatif kasus Covid-19 per tanggal tersebut adalah 4,277,644

In [10]:
latitude_below_0 = df_header[(df_header["Lat"] < 0) & (df_header["Country/Region"] != "Indonesia")]
df_result = df_history_cumsum[[col for col in latitude_below_0["Identifier"]]]
result = df_result.loc["20/01/22"].idxmax()
print(
    f"Negara yang paling tinggi jumlah kumulatif kasus Covidnya per tanggal tersebut adalah '{result.split('-')[0]}' dengan jumlah kasus kumulatif Covid-19 {formatter(df_result.loc['20/01/22'][result])}")
print(
    f"Sementara negara Indonesia paling tinggi jumlah kumulatif kasus Covid-19 per tanggal tersebut adalah {formatter(df_history_cumsum.loc['20/01/22']['Indonesia-nan'])}")

Negara yang paling tinggi jumlah kumulatif kasus Covidnya per tanggal tersebut adalah 'Brazil' dengan jumlah kasus kumulatif Covid-19 23,595,178
Sementara negara Indonesia paling tinggi jumlah kumulatif kasus Covid-19 per tanggal tersebut adalah 4,277,644


# Question 2
Buatlah fungsi untuk menghitung jumlah kasus harian Covid-19! (Hint: Kamu bisa menggunakan .diff() untuk menghitung perbedaan tiap baris data/menghitung jumlah kasus harian dan .transpose() untuk mengubah kolom menjadi baris atau sebaliknya)

# Answer :
- Using index locator we will retrieve the historical data
- doing transpose to pivot the frame
- parsing the index to be the date.
- using col `Identifier` to set column name
- then use diff to calculate the delta/difference between the date on each Country.


In [11]:
def pivot(df):
    _df = df.iloc[:, 4:-1].T
    _df.index = pd.to_datetime(df_history_cumsum.index)
    _df.columns = df["Identifier"].values
    return _df.diff()

pivot(df)

,Afghanistan-nan,Albania-nan,Algeria-nan,Andorra-nan,Angola-nan,Antarctica-nan,Antigua and Barbuda-nan,Argentina-nan,Armenia-nan,Australia-Australian Capital Territory,...,Uruguay-nan,Uzbekistan-nan,Vanuatu-nan,Venezuela-nan,Vietnam-nan,West Bank and Gaza-nan,Winter Olympics 2022-nan,Yemen-nan,Zambia-nan,Zimbabwe-nan
2020-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
2020-01-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-15,176.0,187.0,405.0,84.0,41.0,0.0,11.0,18573.0,1307.0,557.0,...,6120.0,302.0,0.0,1030.0,31814.0,4681.0,4.0,20.0,291.0,0.0
2022-02-16,97.0,449.0,424.0,91.0,13.0,0.0,53.0,17034.0,1774.0,526.0,...,6640.0,296.0,0.0,1065.0,34737.0,3648.0,2.0,8.0,285.0,610.0
2022-02-17,154.0,361.0,375.0,70.0,17.0,0.0,0.0,16650.0,1723.0,522.0,...,5517.0,241.0,0.0,926.0,36200.0,2853.0,0.0,11.0,319.0,385.0
2022-02-18,184.0,300.0,316.0,67.0,20.0,0.0,5.0,15389.0,1417.0,332.0,...,7179.0,272.0,0.0,979.0,42439.0,2928.0,1.0,13.0,290.0,432.0


# Question 3
Pada tanggal berapa puncak tertinggi kasus harian Covid-19 di Indonesia? bandingkan dengan tanggal 20 Januari 2022, apakah mengalami penurunan atau kenaikan jumlah kasus?

## Answer :
After doing some query in this section, we can conclude that the peak case count is 5,149,021 and the peak case on 20 January 2022 is 4,277,644.
so we can conclude that there is an increase of Covid-19 Cases after 20 January 2022.

In [12]:
indonesia_history = df_history_cumsum["Indonesia-nan"]
print("Jumlah kasus harian Covid-19 di Indonesia pada tanggal 20 Januari 2022 adalah", formatter(indonesia_history.loc["20/01/22"]))
print("Sementara Jumlah kasus harian Covid-19 di Indonesia Tertinggi adalah", formatter(indonesia_history.max()))

Jumlah kasus harian Covid-19 di Indonesia pada tanggal 20 Januari 2022 adalah 4,277,644
Sementara Jumlah kasus harian Covid-19 di Indonesia Tertinggi adalah 5,149,021


# Question 4
Gunakan for loop dan conditional if untuk membuat sebuah list yang berisikan jumlah kasus harian Covid-19 dengan ketentuan hanya mengambil data Indonesia saja dan jumlah kasus di bawah 10000. Berapa jumlah hari dengan kasus di bawah 10000?

## Answer :
from the query result, we can conclude that there are 99 days with less than 10,000 cases.

In [13]:
result = [i for i in df_history_cumsum["Indonesia-nan"] if i < 10_000]

print(f"Jumlah hari dengan kasus di bawah 10.000 adalah {len(result)}")

Jumlah hari dengan kasus di bawah 10.000 adalah 99
